## Here is the python code for the 'Generate_monthly_bill' and few test output samples along with the thought process of makeing the work flow smoother

In [42]:
from datetime import datetime # import the datetime class to handle date related data like parsing, camparing and calculating
from collections import defaultdict # a dictionary subclass that provides default values for nonexistent keys. Helps in aggregating quantities and amounts.
import calendar # calander module is called to dittermine the number of days in a given month

def generate_monthly_bill(item_list: list, target_month: str) -> dict: # define a function that takes the 'item_list and target_months'
    
    # A docstring below describing what the function does, its parameters, and what it returns.
    """ 
    Generates a monthly bill for active items during the specified month.
    
    Arguments:
        item_list (list): List of items with details including start/stop dates, qty, rate.
        target_month (str): Month for billing in the format 'YYYY-MM'.
        
    Returns:
        dict: Contains a list of line items and the total revenue for the month.
    """
    
    # Parse target_month into datetime range
    # set date range for the target month
    year, month = map(int, target_month.split('-')) # Splits "YYYY-MM" into integers: year and month.
    month_start = datetime(year, month, 1) # Creates a datetime for the first day of the month.
    last_day = calendar.monthrange(year, month)[1] # Finds the last day of the month
    month_end = datetime(year, month, last_day) # Creates a datetime for the last day of the month (yyyy-mm-day'last_day')

    # Helper function to safely convert to number
    def to_number(value, is_int=False): # this checks if the value is not 'int' and the convert it to the int data type
        try: 
            return int(value) if is_int else float(value) # returning the int value, if not int then return the float value
        except (ValueError, TypeError): # excepts the value and the type error here and returns the number zero
            return 0 # safely converts strings or other types to numbers (int or float). Returns 0 if invalid.

    # Container for grouped billing data
    groups = defaultdict(lambda: {"qty": 0, "amount": 0.0}) # here we are initializing a dictionary where each key maps to a dict holding quantity and amount totals.

    # Process each item - loop through all 'item'
    for item in item_list: # parses item’s start and stop dates
        item_start = datetime.strptime(item["start_date"], "%Y-%m-%d")
        item_end = datetime.strptime(item["stop_date"], "%Y-%m-%d")

        # Skip items not active during target month
        if item_end < month_start or item_start > month_end: # if item_end date is less then month_start and vise-versa then skip & continue fall into a next block of code
            continue

        # Calculate overlap with billing month
        billing_start = max(item_start, month_start) 
        billing_end = min(item_end, month_end) # Finds the intersection (active range) between item and billing month.   
        active_days = (billing_end - billing_start).days + 1 # Computes the number of active days for this item 
        total_days = (month_end - month_start).days + 1 # and total days in the month.
        billing_period_str = f"{billing_start.date()} to {billing_end.date()}" # this will keep a eye on billing period calculation, like billing start days and billing end days
        # this crete a formated string/readble string
        
        # Normalize and calculate
        # fetches and converts the item code, quantity, and rate.
        item_code = item["item_code"]
        qty = to_number(item.get("qty"), is_int=True)
        rate = to_number(item.get("rate"))
        prorated_amount = round(rate * qty * (active_days / total_days), 2) # calculates prorated amount for the item based on how many days it was active.

        # Group by item_code, rate, and billing_period
        key = (item_code, rate, billing_period_str)
        groups[key]["qty"] += qty
        groups[key]["amount"] += prorated_amount # group items by item_code, rate, billing_period_str and sum up quantity and amount

    # Prepare the final result
    line_items = [] # initializing the list of bills and ravanue tracker
    total_revenue = 0.0 # consider the floting values

    for (item_code, rate, billing_period), values in groups.items(): # loop through all group
        amount = round(values["amount"], 2) # Rounds the amount to two decimal places.
        line_items.append({ # it appends all the titles and its values
            "item_code": item_code,
            "rate": rate,
            "qty": values["qty"],
            "amount": amount,
            "billing_period": billing_period,
        })
        total_revenue += amount 

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2) # again we are rounding the value by two decimal places
    }


In [38]:
target_month = "2023-11"  # The month we want the bill for, we can alter is year and month and we will be generating the bill

# Your predefined item_list

item_list = [ # given in the assignment as a sample
{
"idx": 1,
"item_code": "Executive Desk (4*2)",
"sales_description": "Dedicated Executive Desk",
"qty": 10,
"rate": "1000",
"amount": "10000",
"start_date": "2023-11-01",
"stop_date": "2024-10-17",
},
{
"idx": 2,
"item_code": "Executive Desk (4*2)",
"qty": "10",
"rate": "1080",
"amount": "10800",
"start_date": "2024-10-18",
"stop_date": "2025-10-31",
},
{
"idx": 3,
"item_code": "Executive Desk (4*2)",
"qty": 15,
"rate": "1080",
"amount": "16200",
"start_date": "2024-11-01",
"stop_date": "2025-10-31",
},
{
"idx": 4,
"item_code": "Executive Desk (4*2)",
"qty": 5,
"rate": "1000",
"amount": "5000",
"start_date": "2024-11-01",
"stop_date": "2025-10-31",
},
{
"idx": 5,
"item_code": "Manager Cabin",
"qty": 5,
"rate": 5000,
"amount": 25000,
"start_date": "2024-11-01",
"stop_date": "2025-10-31",
},
{
"idx": 6,
"item_code": "Manager Cabin",
"qty": 7,
"rate": "5000",
"amount": 35000,
"start_date": "2024-12-15",
"stop_date": "2025-10-31",
},
{
"idx": 7,
"item_code": "Manager Cabin",
"qty": 10,
"rate": 4600,
"amount": 46000,
"start_date": "2023-11-01",
"stop_date": "2024-10-17",
},
{
"idx": 8,
"item_code": "Parking (2S)",
"qty": 10,
"rate": 1000,
"amount": 10000,
"start_date": "2024-11-01",
"stop_date": "2025-10-31",
},
{
"idx": 9,
"item_code": "Parking (2S)",
"qty": 10,
"rate": 0,
"amount": 0,
"start_date": "2024-11-01",
"stop_date": "2025-10-31",
},
{
"idx": 10,
"item_code": "Executive Desk (4*2)",
"qty": "8",
"rate": "1100",
"amount": "8800",
"start_date": "2024-11-15",
"stop_date": "2025-01-31",
},
{
"idx": 11,
"item_code": "Manager Cabin",
"qty": "3",
"rate": "5200",
"amount": "15600",
"start_date": "2024-10-10",
"stop_date": "2024-11-10",
},
{
"idx": 12,
"item_code": "Conference Table",
"qty": 1,
"rate": "20000",
"amount": "20000",
"start_date": "2024-11-05",
"stop_date": "2024-11-20",
},
{
"idx": 13,
"item_code": "Parking (2S)",
"qty": 5,
"rate": "1000",
"amount": "5000",
"start_date": "2024-11-15",
"stop_date": "2025-02-28",
},
{
"idx": 14,
"item_code": "Reception Desk",
"qty": 2,
"rate": "7000",
"amount": "14000",
"start_date": "2024-11-01",
"stop_date": "2025-03-31",
},
{
"idx": 15,
"item_code": "Reception Desk",
"qty": 1,
"rate": "7000",
"amount": "7000",
"start_date": "2024-11-10",
"stop_date": "2024-11-25",
},
{
"idx": 16,
"item_code": "Breakout Area",
"qty": 3,
"rate": "3000",
"amount": "9000",
"start_date": "2024-01-01",
"stop_date": "2024-01-31",
}
]

# Call the function
bill = generate_monthly_bill(item_list, target_month) # calling the define funcion and passing the item_list and target_month from it

# Output the result
print(bill) # printing the final bill here 

{'line_items': [{'item_code': 'Executive Desk (4*2)', 'rate': 1000.0, 'qty': 10, 'amount': 10000.0, 'billing_period': '2023-11-01 to 2023-11-30'}, {'item_code': 'Manager Cabin', 'rate': 4600.0, 'qty': 10, 'amount': 46000.0, 'billing_period': '2023-11-01 to 2023-11-30'}], 'total_revenue': 56000.0}


# Testing the code with few more inputs

In [44]:
target_month = "2024-11"  # The month you want the bill for

# our predefined item_list (given in the assignment)
item_list = [
    {
        "item_code": "Executive Desk (4*2)",
        "rate": 1000,
        "qty": 10,
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "item_code": "Executive Desk (4*2)",
        "rate": 1080,
        "qty": 10,
        "start_date": "2024-10-18",
        "stop_date": "2025-10-31",
    },
    {
        "item_code": "Executive Desk (4*2)",
        "rate": 1080,
        "qty": 15,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    }
]

# Call the function
bill = generate_monthly_bill(item_list, target_month)

# print the result
print(bill)

{'line_items': [{'item_code': 'Executive Desk (4*2)', 'rate': 1080.0, 'qty': 25, 'amount': 27000.0, 'billing_period': '2024-11-01 to 2024-11-30'}], 'total_revenue': 27000.0}


## It's just a thought in my mind

### We can Convert the item_list to a CSV file and connecting it to your Python function can be an idea(just an opinion), especially in real-world scenarios. This is common practice for data handling as it allows for easier management and updating of the dataset.

## Why we can use a CSV File?

### - Scalability: We can easily add, remove, or update items without modifying the code itself.

### - Maintainability: CSV files are simple, widely understood, and can be edited with Excel or other tools.

### - Data Integration: If this project were to evolve or be integrated with larger systems, working with external files (like CSV) can be a common approach.